# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [1]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import time

BATCH_SIZE = 128
NUM_EPOCHS = 10

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [2]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

Then, we define the model, object function and optimizer that we use to classify.

In [3]:
class SimpleNet(nn.Module):
# TODO:define model
    def __init__(self):
        super(SimpleNet,self).__init__()
        self.conv1 = nn.Sequential(torch.nn.Conv2d(1,64,kernel_size=3,stride=1,padding=1),                                                               torch.nn.ReLU(),
                                   torch.nn.Conv2d(64,128,kernel_size=3,stride=1,padding=1),      
                                   torch.nn.MaxPool2d(stride=2,kernel_size=2),
                                   torch.nn.ReLU(),
                                   torch.nn.BatchNorm2d(128,eps=1e-8,affine=True))     
                                        
        self.dense = nn.Sequential(torch.nn.Linear(14*14*128,1024),
                                   torch.nn.Sigmoid(),
                                   torch.nn.Linear(1024,10))
                                   
    def forward(self,x):
        x = self.conv1(x)
        x = x.view(-1,14*14*128)
        x = self.dense(x)
        return x
    
model = SimpleNet()

# TODO:define loss function and optimiter
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=3e-4)

#使用GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(device)

model.to(device)

cuda:0


SimpleNet(
  (conv1): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): ReLU()
    (5): BatchNorm2d(128, eps=1e-08, momentum=0.1, affine=True, track_running_stats=True)
  )
  (dense): Sequential(
    (0): Linear(in_features=25088, out_features=1024, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=1024, out_features=10, bias=True)
  )
)

Next, we can start to train and evaluate!

In [4]:
# train and evaluate
for epoch in range(NUM_EPOCHS):
    print("Epoch{}/{}".format(epoch,NUM_EPOCHS))
    print("-"*10)
    print("training")
    running_loss = 0.0
    running_correct = 0
    testing_correct = 0
    for images, labels in tqdm(train_loader):
        # TODO:forward + backward + optimize
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _,predicted = torch.max(outputs.data,1)
        optimizer.zero_grad()
        loss = criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        running_correct += torch.sum(predicted == labels.data)
    with torch.no_grad():
        for images, labels in tqdm(test_loader):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _,predicted = torch.max(outputs,1)
            testing_correct += torch.sum(predicted == labels.data)
    
    # evaluate
    # TODO:calculate the accuracy using traning and testing dataset
   
    loss = running_loss / len(train_dataset)
    training_accuracy = running_correct.item() / len(train_dataset)
    testing_accuracy = testing_correct.item() / len(test_dataset)
    
    print("Loss is :{:.4f},Train Accuracy is:{:.2f}%,Test Accuracy is:{:.2f}".format(loss,training_accuracy*100,testing_accuracy*100))

0%|          | 0/468 [00:00<?, ?it/s]Epoch0/10
----------
training
  0%|          | 1/468 [00:00<00:54,  8.57it/s]Loss is :0.0009,Train Accuracy is:96.76%,Test Accuracy is:98.62
Epoch1/10
----------
training
  0%|          | 1/468 [00:00<00:52,  8.95it/s]Loss is :0.0002,Train Accuracy is:99.14%,Test Accuracy is:98.64
Epoch2/10
----------
training
  0%|          | 1/468 [00:00<00:47,  9.83it/s]Loss is :0.0001,Train Accuracy is:99.58%,Test Accuracy is:99.03
Epoch3/10
----------
training
  0%|          | 1/468 [00:00<00:49,  9.37it/s]Loss is :0.0000,Train Accuracy is:99.75%,Test Accuracy is:99.04
Epoch4/10
----------
training
  0%|          | 1/468 [00:00<00:52,  8.87it/s]Loss is :0.0000,Train Accuracy is:99.81%,Test Accuracy is:99.13
Epoch5/10
----------
training
  0%|          | 2/468 [00:00<00:48,  9.70it/s]Loss is :0.0000,Train Accuracy is:99.81%,Test Accuracy is:98.97
Epoch6/10
----------
training
  0%|          | 1/468 [00:00<00:50,  9.28it/s]Loss is :0.0000,Train Accuracy is:99.84%

#### Q5:
Please print the training and testing accuracy.

In [5]:
print("Train Accuracy is:{:.2f}% \nTest Accuracy is:{:.2f}%".format(training_accuracy*100,testing_accuracy*100))

Train Accuracy is:99.84% 
Test Accuracy is:99.13%
